# Auto Encoders

In [1]:
# Importar las librerías
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

In [2]:
# Importar el dataset
movies = pd.read_csv("ml-1m/movies.dat", sep = '::', header = None, engine = 'python', encoding = 'latin-1')
users  = pd.read_csv("ml-1m/users.dat", sep = '::', header = None, engine = 'python', encoding = 'latin-1')
ratings  = pd.read_csv("ml-1m/ratings.dat", sep = '::', header = None, engine = 'python', encoding = 'latin-1')

In [3]:
# Preparar el conjunto de entrenamiento y elconjunto de testing
training_set = pd.read_csv("ml-100k/u1.base", sep = "\t", header = None) #base=trainig
training_set = np.array(training_set, dtype = "int") #para pytorch pasamos el dataframe de pandas a array
test_set = pd.read_csv("ml-100k/u1.test", sep = "\t", header = None) #test=test
test_set = np.array(test_set, dtype = "int") #pasamos el dataframe a formato array

In [4]:
# Obtener el número de usuarios y de películas
nb_users = int(max(max(training_set[:, 0]), max(test_set[:,0])))
nb_movies = int(max(max(training_set[:, 1]), max(test_set[:, 1])))

In [6]:
# Convertir los datos en un array X[u,i] con usuarios 'u' en fila y películas 'i' en columna
def convert(data):
    new_data = []
    for id_user in range(1, nb_users+1):
        id_movies = data[:, 1][data[:, 0] == id_user]
        id_ratings = data[:, 2][data[:, 0] == id_user]
        ratings = np.zeros(nb_movies)
        ratings[id_movies-1] = id_ratings
        new_data.append(list(ratings))
    return new_data

training_set = convert(training_set)
test_set = convert(test_set)

In [7]:
# Convertir los datos a tensores de Torch
training_set = torch.FloatTensor(training_set)
test_set = torch.FloatTensor(test_set)

In [8]:
# Crear la arquitectura de la Red Neuronal
class SAE(nn.Module): #clase que herada de otra clase Module, siendo 'nn' el paquete de Pytorch donde se encuentra
    def __init__(self, ): #constructor de la clase
        super(SAE, self).__init__() #llamaos al constructor del padre usando super(...).__init__()
        
        #deinifmos los AutoEncoders apilados que consistira en codificar los datos de entrada
        #pasar de 20 neuronas a 10 (codificacion) y de 10 a 20 otra vez(decodificacion):
        
        #full connection(fc), capa de entrada con nb_movies numero de neuronas y 
        #con capa oculta 20 caracteristicas-neuronas 
        self.fc1 = nn.Linear(nb_movies, 20) 
        #Otra fc capa oculta que conectara la capa anterior (que tenia 20) y le ponemos una salida
        #de 10 neuronas de salida (codificacion)
        self.fc2 = nn.Linear(20, 10)
        #lo mismo de arriba que conecta las 10 neuronas de arriba con 20 en otra capa oculta que ponemos
        self.fc3 = nn.Linear(10, 20)
        #lo mismo, conectamos las 20 neuronas de salida de fc3 a la capa de salida con nb_movies-neuronas
        self.fc4 = nn.Linear(20, nb_movies)
        
        #funcion de activacion que indicara el umbral para que se activen las neuronas
        self.activation = nn.Sigmoid() 
    
    #funcion donde la info fluira desde la codificacion a la descodificacion
    #self para usar la variables del contructor de la clase y 'x' indicara los datos
    def forward(self, x): 
        x = self.activation(self.fc1(x)) #codificacion pasado por la funcion de activacion
        x = self.activation(self.fc2(x))
        x = self.activation(self.fc3(x))
        x = self.fc4(x) #queremos el valor de la pelicula, ya no hace falta meter la funcion de activacion
        return x

sae = SAE() #creamos un objeto de la clase SAE()
criterion = nn.MSELoss() #criterio de optimizacion(error) usando el error cuadrado medio (MSE) para que el entrenamiento suceda
#optimizador usando RMS(tambien se podría usar el Adam)
#lr=learning rate(tasa de aprendizaje)
#weight_decay para ajustar el lr
optimizer = optim.RMSprop(sae.parameters(), lr = 0.01, weight_decay = 0.5) 

In [9]:
# Entrenar el SAE
nb_epoch = 200
for epoch in range(1, nb_epoch+1): #bucle para recorrer cada la red 200 veces
    
    train_loss = 0 #perdidas para medir la cantidad de errores en el entrenamiento(usando mean)
    s = 0. #Cuantos usuarios calificaron al menos una pelicula, en decimal
    
    for id_user in range(nb_users): #bucle para recorrer los usuarios
        
        #vector de entrada(valoraciones de cada usuario), al dar un vector de un elemento...
        #pero pythorch o keras exige introducir un array con mas de un elemento(variable().unsqueeze(0))
        #añadimos una dimesion 'falsa'
        input = Variable(training_set[id_user]).unsqueeze(0) 
        
        #hacemos una copia de input(el vector de entrada para poder medir al final el error en funcion de la prevision
        target = input.clone() 
        if torch.sum(target.data > 0) > 0: #para calificar usuarios que hayan calificado alguna pelicula (0 seria ninguna)
            
            #salida de la red para los pronosticos pasando por las capas creadas antes en SAE()
            output = sae.forward(input)
            #para ignorar el gradiente descendiente
            target.require_grad = False
            #solo queremos aplicar el gradiente descenente el target a predecir es 0
            output[target == 0] = 0
            loss = criterion(output, target)
            # la media no es sobre todas las películas, sino sobre las que realmente ha valorado
            mean_corrector = nb_movies/float(torch.sum(target.data > 0)+1e-10) 
            loss.backward()
            train_loss += np.sqrt(loss.data*mean_corrector) ## sum(errors) / n_pelis_valoradas
            s += 1.
            optimizer.step()
    print("Epoch: "+str(epoch)+", Loss: "+str(train_loss/s))

Epoch: 1, Loss: tensor(1.7712)
Epoch: 2, Loss: tensor(1.0967)
Epoch: 3, Loss: tensor(1.0534)
Epoch: 4, Loss: tensor(1.0382)
Epoch: 5, Loss: tensor(1.0310)
Epoch: 6, Loss: tensor(1.0265)
Epoch: 7, Loss: tensor(1.0238)
Epoch: 8, Loss: tensor(1.0219)
Epoch: 9, Loss: tensor(1.0209)
Epoch: 10, Loss: tensor(1.0195)
Epoch: 11, Loss: tensor(1.0190)
Epoch: 12, Loss: tensor(1.0185)
Epoch: 13, Loss: tensor(1.0180)
Epoch: 14, Loss: tensor(1.0174)
Epoch: 15, Loss: tensor(1.0174)
Epoch: 16, Loss: tensor(1.0168)
Epoch: 17, Loss: tensor(1.0170)
Epoch: 18, Loss: tensor(1.0166)
Epoch: 19, Loss: tensor(1.0165)
Epoch: 20, Loss: tensor(1.0160)
Epoch: 21, Loss: tensor(1.0160)
Epoch: 22, Loss: tensor(1.0159)
Epoch: 23, Loss: tensor(1.0159)
Epoch: 24, Loss: tensor(1.0159)
Epoch: 25, Loss: tensor(1.0159)
Epoch: 26, Loss: tensor(1.0155)
Epoch: 27, Loss: tensor(1.0154)
Epoch: 28, Loss: tensor(1.0152)
Epoch: 29, Loss: tensor(1.0130)
Epoch: 30, Loss: tensor(1.0120)
Epoch: 31, Loss: tensor(1.0097)
Epoch: 32, Loss: 

In [10]:
# Evaluar el conjunto de test en nuestro SAE
test_loss = 0
s = 0.
for id_user in range(nb_users):
    input = Variable(training_set[id_user]).unsqueeze(0)
    target = Variable(test_set[id_user]).unsqueeze(0)
    if torch.sum(target.data > 0) > 0:
        output = sae.forward(input)
        target.require_grad = False
        output[target == 0] = 0
        loss = criterion(output, target)
        # la media no es sobre todas las películas, sino sobre las que realmente ha valorado
        mean_corrector = nb_movies/float(torch.sum(target.data > 0)+1e-10) 
        test_loss += np.sqrt(loss.data*mean_corrector) ## sum(errors) / n_pelis_valoradas
        s += 1.
print("Test Loss: "+str(train_loss/s))


Test Loss: tensor(1.8734)
